<a href="https://colab.research.google.com/github/jdmedinatobon/proyectoMachineLearning/blob/master/CrearArchivoDescriptores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install mahotas

ERROR: Could not find a version that satisfies the requirement unzip (from versions: none)
ERROR: No matching distribution found for unzip


In [1]:
import os
import numpy as np
import nibabel as nib
import csv
import mahotas as mh

%tensorflow_version 2.x
import tensorflow as tf
import time

TensorFlow 2.x selected.


In [0]:
#Se descomprime el archivo con los datos preprocesados
!unzip PruebaDatos.zip #En verdad no es este archivo.

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Archive:  PruebaDatos.zip
   creating: PruebaDatos/
   creating: PruebaDatos/func/
 extracting: PruebaDatos/func/sub-01_task-LTM_run-1_bold.nii.gz  
  inflating: PruebaDatos/func/sub-01_task-LTM_run-1_events.tsv  
Found GPU at: /device:GPU:0


In [0]:
pathArchivoDatosPreprocesados = "PruebaDatos" #Ruta del archivo raiz de los datos preprocesados

sub = [0, 0]
run = 1

def darNombreArchivoImagenes(pSub, pRun):
  return "PruebaDatos/func/sub-" + str(pSub[0]) + str(pSub[1]) + "_task-LTM_run-" + str(pRun) + "_bold.nii.gz"

def darNombreArchivoEtiquetas(pSub, pRun):
  return "PruebaDatos/func/sub-" + str(pSub[0]) + str(pSub[1]) + "_task-LTM_run-" + str(pRun) + "_events.tsv"

def obtenerImagenes(pImagenes):
  
  imagenes = np.zeros((64,64,30,30))

  for indice in range(0,15):
    imagenes[:,:,:,2*indice] = pImagenes[:,:,:,1+indice*13]
    imagenes[:,:,:,2*indice+1] = pImagenes[:,:,:,2+indice*13]

  return imagenes

#object -> 0
#place -> 1
#face -> 2
def leerEtiquetas(pArchivoEtiquetas):
  etiquetas = []

  with open(pArchivoEtiquetas) as tsvfile:
    reader = csv.DictReader(tsvfile, dialect='excel-tab')
    for row in reader:
      
      etiqueta = row['trial_type']

      if(etiqueta == 'object'):

        etiquetas.append(0)
        etiquetas.append(0)
      elif(etiqueta == 'place'):

        etiquetas.append(1)
        etiquetas.append(1)
      elif(etiqueta == 'face'):

        etiquetas.append(2)
        etiquetas.append(2)
      else:
        print("Error en el formato.")


  return etiquetas

def importarDatos():
  datosPreprocesados = np.zeros((64,64,30,1800))
  etiquetasPreprocesados = np.zeros(1800)

  contador = 0
  for s1 in range(1,2):#(1,11)

    if(s1 == 10):
      sub[0] = 1
      sub[1] = 0
    else:
      sub[1] = s1

      for r in range(1,2):#(1,7)
        run = r
        archivoImagenes = darNombreArchivoImagenes(sub, run)
        archivoEtiquetas = darNombreArchivoEtiquetas(sub, run)

        imagenes = nib.load(archivoImagenes).get_fdata()
        etiquetas = leerEtiquetas(archivoEtiquetas)

        muestras = obtenerImagenes(imagenes)

        for indice in range(0,30):
            datosPreprocesados[:,:,:,30*contador+indice] = muestras[:,:,:,indice]
            etiquetasPreprocesados[30*contador+indice] = etiquetas[indice]

        contador+=1

    return datosPreprocesados, etiquetasPreprocesados.astype(int)

In [36]:
print("Importando datos...")
test, testet=importarDatos()
print("Se importaron correctamente.")

Importando datos...
Se importaron correctamente.


In [0]:
def calcularDescriptores(pImagenes):
  #Faltan los descriptores del histograma
  return mh.features.haralick(pImagenes, ignore_zeros=True, preserve_haralick_bug=False, compute_14th_feature=False, return_mean=True, return_mean_ptp=False, use_x_minus_y_variance=False, distance=1)

def darTextoIntroduccion():
  texto = "Este archivo incluye los descriptores calculados a partir de los datos preprocesados. \nLos datos brutos fueron obtenidos del \
dataset de imágenes de resonancia magnética funcional que se puede encontrar en https://openneuro.org/datasets/ds001497/versions/1.0.1.\n\
A estos datos se les realizó un preprocesamiento utilizando fmriprep (ESTO TOCA VER SI SI SIRVE AL FIN) y finalmente se calcularon los \
descriptores.\nEstos descriptores son (EL NUMERO QUE SEA) e incluyen:\nEnergía, etc.... (AGREGAR LOS QUE FALTAN Y ORDENARLOS CORRECTAMENTE).\n\
Finalmente, cada muestra incluye a la clase a la cual pertenece. Existen 3 clases y se indican con un número de 0 a 2 que corresponden a:\n\
objeto -> 0\n\
lugar -> 1\n\
rostro -> 2\n\
energia;kurtosis;skewness;....;clase AGREGAR TODAS LAS VARIABLES ACA EN ORDEN SIGUIENDO LA CONVENCIÓN DE SEPARARLOS CON ;"

  return texto

def darLinea(pDescriptores, pEtiqueta):
  separador = ";"
  muestra = []
  return separador.join(pDescriptores.astype(str)) + separador + str(pEtiqueta)

def crearArchivoDescriptores():
  escala = 256

  print("Generando archivo de descriptores...")
  archivo = open("datosDescriptores.txt", "w+")

  intro = darTextoIntroduccion()
  archivo.write(intro)
  archivo.write("\n")

  start = time.time()
  for i in range(0,30):
    test2 = test[:,:,:,i]
    test2 = escala*(test2/test2.max())
    test2 = test2.astype(int)

    descriptores = calcularDescriptores(test2)
    linea = darLinea(descriptores, testet[i])
    archivo.write(linea)
    archivo.write("\n")
    
  print("Archivo generado exitosamente.")
  print("Tiempo: {} segundos".format(time.time()-start))

  archivo.close()

In [68]:
crearArchivoDescriptores()

Generando archivo de descriptores...
Archivo generado exitosamente.
Tiempo: 3.7872633934020996 segundos


In [29]:
print(testet)

[0 0 2 ... 0 0 0]


In [28]:
testet

array([0, 0, 2, ..., 0, 0, 0])

In [0]:
a = ["1", "2", "3"]
b = 4

In [32]:
a[-1] + str(b)

'34'

In [41]:
darTextoIntroduccion()

'Este archivo incluye los descriptores calculados a partir de los datos preprocesados. Los datos brutos fueron obtenidos del   dataset  de imágenes de resonancia magnética funcional que se puede encontrar en https://openneuro.org/datasets/ds001497/versions/1.0.1.   A estos datos se les realizó un preprocesamiento utilizando fmriprep (ESTO TOCA VER SI SI SIRVE AL FIN) y finalmente se calcularon los   descriptores. Estos descriptores son (EL NUMERO QUE SEA) e incluyen: \n Energía, etc.... (AGREGAR LOS QUE FALTAN Y ORDENARLOS CORRECTAMENTE).'